# A Program to
> 1. Swap two numbers

In [1]:
x = 10
y = 20
print("x = {}, y = {}".format(x, y))

x = 10, y = 20


In [2]:
x, y = y, x

In [3]:
print("x = {}, y = {}".format(x, y))

x = 20, y = 10


# Write a program to
>1. Generate Fibonacci Numbers

In [4]:
def fib(num): 
    if num < 0: 
        print("Invalid") 
    elif num == 1: 
        return 0 
    elif num == 2: 
        return 1
    else: 
        return fib(num-1) + fib(num-2) 

In [5]:
fib(4) # 1. fibonacci

2

>2. Create a list using even and odd indices from two lists

In [6]:
import random
lst1 = []
for i in range(4):
    lst1.append(fib(random.randint(2, 20)))
lst1

[34, 2584, 3, 8]

In [7]:
lst2 = []
for i in range(5):
    lst2.append(fib(random.randint(2, 20)))
lst2

[144, 34, 144, 89, 3]

In [8]:
lst3 = lst1[1::2] + lst2[::2] # 2. odd-even merge

>3. Remove an element from an index and add it to some other index

In [9]:
list_of_params = ['p1', 'p2', 'p3']
list_of_params

['p1', 'p2', 'p3']

In [10]:
list_of_params.remove('p2')

In [11]:
list_of_params

['p1', 'p3']

In [12]:
list_of_params.insert(0, 'p2') 
list_of_params # 3. remove & insert at given position

['p2', 'p1', 'p3']

>4. Initialize dictionary with default values

In [13]:
val = "Default"

In [14]:
x = dict()
x

{}

In [15]:
for param in list_of_params:
    x.setdefault(param, val) # 4. default init

In [16]:
x

{'p2': 'Default', 'p1': 'Default', 'p3': 'Default'}

>5. Merge two dictionries

In [17]:
y = {'p4': 'Inserted'}
y

{'p4': 'Inserted'}

In [18]:
z = {**x, **y}
z # 5. merge

{'p2': 'Default', 'p1': 'Default', 'p3': 'Default', 'p4': 'Inserted'}

# Data Preprocessing

In [47]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt
import km
import kmapper
from sklearn import ensemble

In [33]:
cancer = load_breast_cancer()
df = pd.DataFrame(np.c_[cancer['data'], cancer['target']],
                  columns= np.append(cancer['feature_names'], ['target']))

In [35]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0.0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0.0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0.0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0.0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0.0


In [59]:
feature_names = [c for c in df.columns if c not in ["id", "target"]]
df["target"] = df["target"].apply(lambda x: 1 if x == "M" else 0)
X = np.array(df[feature_names].fillna(0)) # quick and dirty imputation
y = np.array(df["target"])

# We create a custom 1-D lens with Isolation Forest
model = ensemble.IsolationForest(random_state=1729)
model.fit(X)
lens1 = model.decision_function(X).reshape((X.shape[0], 1))

# We create another 1-D lens with L2-norm
mapper = kmapper.KeplerMapper(verbose=3)
lens2 = mapper.fit_transform(X, projection="l2norm")

# Combine both lenses to create a 2-D [Isolation Forest, L^2-Norm] lens
lens = np.c_[lens1, lens2]

# Create the simplicial complex
graph = mapper.map(lens, 
                   X,
                   clusterer=kmapper.cluster.KMeans(n_clusters=2, random_state=1618033))

# Visualization
mapper.visualize(graph, 
                 path_html="breast-cancer.html", 
                 title="Wisconsin Breast Cancer Dataset", 
                 custom_tooltips=y)

KeplerMapper(verbose=3)
..Composing projection pipeline of length 1:
	Projections: l2norm
	Distance matrices: False
	Scalers: MinMaxScaler(copy=True, feature_range=(0, 1))
..Projecting on data shaped (569, 30)

..Projecting data using: l2norm

..Scaling with: MinMaxScaler(copy=True, feature_range=(0, 1))

Mapping on data shaped (569, 30) using lens shaped (569, 2)

Minimal points in hypercube before clustering: 2
Creating 100 hypercubes.
Cube_0 is empty.

Cube_1 is empty.

Cube_2 is empty.

Cube_3 is empty.

   > Found 2 clusters in hypercube 4.
Cube_5 is empty.

   > Found 2 clusters in hypercube 6.
Cube_7 is empty.

Cube_8 is empty.

   > Found 2 clusters in hypercube 9.
Cube_10 is empty.

Cube_11 is empty.

Cube_12 is empty.

Cube_13 is empty.

Cube_14 is empty.

Cube_15 is empty.

   > Found 2 clusters in hypercube 16.
   > Found 2 clusters in hypercube 17.
   > Found 2 clusters in hypercube 18.
   > Found 2 clusters in hypercube 19.
Cube_20 is empty.

   > Found 2 clusters in hype

'<!DOCTYPE html>\n<html>\n\n<head>\n  <meta charset="utf-8">\n  <meta name="generator" content="KeplerMapper">\n  <title>Wisconsin Breast Cancer Dataset | KeplerMapper</title>\n\n  <link rel="icon" type="image/png" href="http://i.imgur.com/axOG6GJ.jpg" />\n\n  <link href=\'https://fonts.googleapis.com/css?family=Roboto+Mono:700,300\' rel=\'stylesheet\' type=\'text/css\'>\n  <style>* {\n  margin: 0;\n  padding: 0;\n}\n\nhtml, body {\n  height: 100%;\n}\n\nbody {\n  font-family: "Roboto Mono", "Helvetica", sans-serif;\n  font-size: 14px;\n}\n\n#logo {\n  position: absolute;\n  right: 00px;\n  top: 0px;\n  width: 90px;\n  height: 90px;\n  z-index: 999999;\n}\n\n#display {\n  color: #95A5A6;\n  background: #212121;\n}\n\n#print {\n  color: #000;\n  background: #FFF;\n}\n\nh1 {\n  font-size: 21px;\n  font-weight: 300;\n  font-weight: 300;\n}\n\nh2 {\n  font-size: 18px;\n  padding-bottom: 20px;\n  font-weight: 300;\n}\n\nh3 {\n  font-size: 14px;\n  font-weight: 700;\n  text-transform: upperc

In [60]:
!pip install jupyterthemes

     |████████████████████████████████| 7.0MB 2.2MB/s eta 0:00:01     |█████████████████████████▌      | 5.6MB 2.7MB/s eta 0:00:01
     |████████████████████████████████| 9.4MB 3.7MB/s eta 0:00:01
     |████████████████████████████████| 51kB 3.6MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 4.2MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 112kB 3.1MB/s eta 0:00:01


  ERROR: Command errored out with exit status 1:
   command: /Users/ramanshgrover/anaconda3/bin/python /Users/ramanshgrover/anaconda3/lib/python3.6/site-packages/pip/_vendor/pep517/_in_process.py build_wheel /var/folders/p1/kct7rr3j64v80xtxz5rp5d6w0000gn/T/tmprt8f4v8p
       cwd: /private/var/folders/p1/kct7rr3j64v80xtxz5rp5d6w0000gn/T/pip-install-_yl2v41d/argon2-cffi
  Complete output (26 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.macosx-10.7-x86_64-3.6
  creating build/lib.macosx-10.7-x86_64-3.6/argon2
  copying src/argon2/__init__.py -> build/lib.macosx-10.7-x86_64-3.6/argon2
  copying src/argon2/low_level.py -> build/lib.macosx-10.7-x86_64-3.6/argon2
  copying src/argon2/_ffi_build.py -> build/lib.macosx-10.7-x86_64-3.6/argon2
  copying src/argon2/_password_hasher.py -> build/lib.macosx-10.7-x86_64-3.6/argon2
  copying src/argon2/exceptions.py -> build/lib.macosx-10.7-x86_64-3.6/argon2
  copying src/argon2/_legacy.py -> bu